Use macro indicators for reversal or volatility / dislocation: implied volatility, realized volatility, return dispersion, pairwise correlation.

In [1]:
def get_outcome(px_close_df):
    best_sharpe_ratios = dict()
    for ticker in px_close_df.columns:
        best_sharpe_ratios[ticker] = get_max_sharpe_ratio(ticker, px_close_df)
    return best_sharpe_ratios

In [ ]:
def get_max_sharpe_ratio(ticker, px_close_df):
    max_sharpe_ratio = float('-inf')
    witnessing_net_returns = None
    witnessing_look_back_periods = None
    witnessing_initial_cash = None
    
    for num_look_back_periods in (1, 2, 3, 4, 5, 6, 12, 18):
        (buy_time, sell_time) = get_holding_window_for_ticker(
            ticker, px_close_df, num_look_back_periods)
        for initial_cash in (0.10, 0.50, 1.00, 5.00, 10.00, 50.00, 100.00):
            strat_holdings_df = get_in_sample_trade_outcomes(ticker, px_close_df, buy_time, sell_time,
                                                             initial_cash)
            net_returns = get_net_returns(ticker, px_close_df, num_look_back_periods, initial_cash,
                                          strat_holdings_df)
            sharpe_ratio = net_returns.mean() / net_returns.std() * np.sqrt(365 * 6)
            if sharpe_ratio > max_sharpe_ratio:
                max_sharpe_ratio = sharpe_ratio
                witnessing_net_returns = net_returns
                witnessing_look_back_periods = num_look_back_periods
                witnessing_initial_cash = initial_cash
    
    return {
        "max_sharpe_ratio": max_sharpe_ratio,
        "witnessing_net_returns": witnessing_net_returns,
        "witnessing_look_back_periods": witnessing_look_back_periods,
        "witnessing_initial_cash": witnessing_initial_cash,
    }

In [ ]:
trade_outcomes 